In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.animation as animation
import math
%matplotlib qt

First we have our state: $\vec z = \begin{bmatrix} x, y, \theta, \delta \end{bmatrix}^T$
We also have our control input: $\begin{bmatrix} v, \varphi \end{bmatrix}^T$

For our kinematic model we use the CG kinematic bycicle model in $\mathbb{R}$

$$ \dot z =\begin{bmatrix} \dot x \\ \dot y \\ \dot \theta \\ \dot \delta \end{bmatrix} = \begin{bmatrix} v \cos{\theta + \beta} \\ v \sin{\theta + \beta} \\ \frac{v \cos{\beta} \tan{\delta}}{L} \\ \varphi \end{bmatrix}$$

We can discretize the model in state space form like the following: 
$$ \begin{bmatrix} x_{t+1} \\ y_{t+1} \\ \theta_{t+1} \\ \delta_{t+1} \end{bmatrix} = \begin{bmatrix} x_t + (v_t \cos{\theta_t + \beta_t}) \\ y_t + (v_t \sin{\theta_t + \beta_t}) \\ \theta_t + (\frac{v_t\cos{\beta_t}\tan{\delta_t}}{L}) \\ \delta_t + \varphi_t \end{bmatrix}$$

This model underlies a no slip condition. Meaning there is no slip at the wheels.

We enhance the kinematich model by more state parameters like velocity and acceleration



$\vec z = \begin{bmatrix} x, y, \dot x, \dot y, \ddot x , \ddot y, \theta, \delta \end{bmatrix}^T$
$\vec u = \begin{bmatrix} \ddot x, \ddot y, \varphi \end{bmatrix}^T$

$$ \dot z =\begin{bmatrix} \dot x \\ \dot y \\ \ddot x \\ \ddot y \\  \dot \theta \\ \dot \delta \end{bmatrix} = \begin{bmatrix} v \cos{\theta + \beta} \\ v \sin{\theta + \beta} \\ 0 \\ 0 \\ \frac{v \cos{\beta} \tan{\delta}}{L} \\ 0 \end{bmatrix} + \begin{bmatrix} 0 & 0 & 0 \\ 0 & 0 & 0 \\ 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 0 \\ 0 & 0 & 1\end{bmatrix} \begin{bmatrix} \ddot x\\ \ddot y\\ \varphi \end{bmatrix} $$

In [2]:
def get_beta(delta, lr, L): 
    return np.arctan((lr*np.tan(delta))/L)

For the first test we will use a constant velocity model

In [3]:
def f(z, u, step): 
    # z = [x,y,x_dot, y_dot, theta, delta]
    beta = get_beta(z[5],lr, L)

    x_dot_next = z[2] + (u[0] * step)
    y_dot_next = z[3] + (u[1] * step)
    
    velocity = np.linalg.norm((x_dot_next, y_dot_next))
    x_next = z[0] + ((velocity*np.cos(z[4] + beta)) * step)
    y_next = z[1] + ((velocity*np.sin(z[4] + beta)) * step)
    theta_next = z[4] + (((velocity*np.cos(beta)*np.tan(z[5]))/L) * step)
    delta_next = z[5] + (u[2] * step)
    
    return np.array([x_next, y_next,x_dot_next, y_dot_next, theta_next, delta_next], dtype=object)  

In [4]:
velocity = 1
lr = 0.5
L = 1.5
step = 0.01
Ts = np.arange(0, 100, step)
#steering = np.sort(np.random.uniform(-2*np.pi + 0.5, 2*np.pi - 0.5, Ts.shape))
#steering = np.array([np.pi if t < Ts.max()/2 else 2*np.pi for t in Ts ])
#steering = np.array([np.pi for t in Ts ])

u = [1, 1, 0]

z = [np.array([0,0,0,0,0,1])]

for i,t in enumerate(Ts): 
    z.append(f(z[i],u, step))
z = np.array(z, dtype=object)

In [5]:
fig = plt.figure()
ax = fig.add_subplot()
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_xlim(-40, 40)
ax.set_ylim(-40, 40)

plt.plot(z[:,0], z[:,1], color="black", alpha = 0.5)
plt.grid(color='0.95')

def func(num):
    x = z[num, 0]
    y = z[num, 1]
    car.set_data(x,y)
    
    


car, = plt.plot([z[0,0]], [z[0,1]], c='r', marker='o')

ani = animation.FuncAnimation(fig, func, frames=len(z), interval=200, blit=False)

plt.show()